# COVID19 - District Region

In [158]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",1)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.init(object_store_memory=1*GB,memory=1*GB,\
         driver_object_store_memory=500*MB,num_gpus=5,num_cpus=5, 
         ignore_reinit_error=True) # , include_webui=False, ignore_reinit_error=True)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-08-01 16:04:59,898	INFO resource_spec.py:212 -- Starting Ray with 0.98 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-01 16:05:00,229	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


ObjectID(90e0fcd01a188068ffffffff010000c801000000)

In [159]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [160]:
# Download data
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

Baixando arquivos brasil.io...
(pid=93107) ray.get_gpu_ids(): 4
(pid=93107) CUDA_VISIBLE_DEVICES: 4


In [161]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv")
dfSP
dfSP.query('DRS == "{}"'.format('DRS 01 - Grande São Paulo'))

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,DRS
3499,2020-07-31,SP,Arujá,city,1079,59,126,True,89824.0,3503901.0,1201.23798,0.0547,DRS 01 - Grande São Paulo
3500,2020-07-30,SP,Arujá,city,1058,59,125,False,89824.0,3503901.0,1177.85892,0.0558,DRS 01 - Grande São Paulo
3501,2020-07-29,SP,Arujá,city,1026,59,124,False,89824.0,3503901.0,1142.23370,0.0575,DRS 01 - Grande São Paulo
3502,2020-07-28,SP,Arujá,city,989,59,123,False,89824.0,3503901.0,1101.04204,0.0597,DRS 01 - Grande São Paulo
3503,2020-07-27,SP,Arujá,city,963,56,122,False,89824.0,3503901.0,1072.09654,0.0582,DRS 01 - Grande São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56367,2020-03-30,SP,Vargem Grande Paulista,city,2,1,5,False,52597.0,3556453.0,3.80250,0.5000,DRS 01 - Grande São Paulo
56368,2020-03-27,SP,Vargem Grande Paulista,city,2,1,4,False,52597.0,3556453.0,3.80250,0.5000,DRS 01 - Grande São Paulo
56369,2020-03-26,SP,Vargem Grande Paulista,city,1,1,3,False,52597.0,3556453.0,1.90125,1.0000,DRS 01 - Grande São Paulo
56370,2020-03-25,SP,Vargem Grande Paulista,city,0,1,2,False,52597.0,3556453.0,NaN,0.0000,DRS 01 - Grande São Paulo


In [162]:
# Model

In [163]:
# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")
DRS

['DRS 09 - Marília',
 'DRS 15 - São José do Rio Preto',
 'DRS 14 - São João da Boa Vista',
 'DRS 06 - Bauru',
 'DRS 16 - Sorocaba',
 'DRS 11 - Presidente Prudente',
 'DRS 05 - Barretos',
 'DRS 13 - Ribeirão Preto',
 'DRS 02 - Araçatuba',
 'DRS 07 - Campinas',
 'DRS 03 - Araraquara',
 'DRS 10 - Piracicaba',
 'DRS 17 - Taubaté',
 'DRS 08 - Franca',
 'DRS 01 - Grande São Paulo',
 'DRS 12 - Registro',
 'DRS 04 - Baixada Santista']

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

In [164]:
#objective function Odeint solver
from scipy.integrate import odeint

#objective function Odeint solver
def lossOdeint(point, data, death, s_0, e_0, a_0, i_0, r_0, d_0, startNCases, ratioRecovered, weigthCases, weigthRecov):
    size = len(data)
    beta, beta2, sigma, sigma2, sigma3, gamma, b, mu = point
    def SEAIRD(y,t):
        S = y[0]
        E = y[1]
        A = y[2]
        I = y[3]
        R = y[4]
        D = y[5]
        p=0.2
        # beta2=beta
        y0=-(beta2*A+beta*I)*S-mu*S #S
        y1=(beta2*A+beta*I)*S-sigma*E-mu*E #E
        y2=sigma*E*(1-p)-gamma*A-mu*A #A
        y3=sigma*E*p-gamma*I-sigma2*I-sigma3*I-mu*I#I
        y4=b*I+gamma*A+sigma2*I-mu*R #R
        y5=(-(y0+y1+y2+y3+y4)) #D
        return [y0,y1,y2,y3,y4,y5]

    y0=[s_0,e_0,a_0,i_0,r_0,d_0]
    tspan=np.arange(0, size, 1)
    res=odeint(SEAIRD,y0,tspan) #,hmax=0.01)

    l1=0
    l2=0
    l3=0
    tot=0

    for i in range(0,len(data.values)):
        if data.values[i]>startNCases:
            l1 = l1+(res[i,3] - data.values[i])**2
            l2 = l2+(res[i,5] - death.values[i])**2
            newRecovered=min(1e6,data.values[i]*ratioRecovered)
            l3 = l3+(res[i,4] - newRecovered)**2
            tot+=1
    l1=np.sqrt(l1/max(1,tot))
    l2=np.sqrt(l2/max(1,tot))
    l3=np.sqrt(l3/max(1,tot))
    
    #weight for cases
    u = weigthCases  #Brazil US 0.1
    w = weigthRecov
    #weight for deaths
    v = max(0,1. - u - w)
    
    return u*l1 + v*l2 + w*l3

In [165]:
from IPython.display import display, HTML, Markdown

paramOpt=1
changeCSV=False
adjustParam=False
paramSave=False
selectDRS=True

In [166]:
if paramOpt==0:
    paramFile="data/param.csv"
    version = "1"
    model = "ManualIC"
    
if paramOpt==1:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version = "106"
    model = "YaboxIC"
    
if paramOpt==2:
    paramFile="data/param_optimized_Yabox_HistMin-Copy1.csv"
    version = "106"
    model = "YaboxICAdjusted"

In [167]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-18,200,7840120,136,0,136,300,196,117,0.15,0.4950,0.0081
1,DRS 02 - Araçatuba,2020-03-29,200,282699,182,0,182,0,51,5,0.15,0.6257,0.0362
2,DRS 03 - Araraquara,2020-03-30,200,377711,534,0,534,0,102,93,0.15,0.5056,0.0034
3,DRS 04 - Baixada Santista,2020-04-01,200,718013,270,0,270,0,281,155,0.15,0.4062,0.0124
4,DRS 05 - Barretos,2020-04-03,200,147064,883,0,883,0,14,105,0.15,0.3535,0.0211
5,DRS 06 - Bauru,2020-04-01,200,717995,263,0,263,0,201,59,0.15,0.5443,0.0304
6,DRS 07 - Campinas,2020-03-29,200,1865626,602,0,602,0,9,20,0.15,0.5461,0.0282
7,DRS 08 - Franca,2020-04-01,200,237187,37,0,37,0,26,3,0.15,0.4088,0.0257
8,DRS 09 - Marília,2020-03-30,200,456106,292,0,292,0,241,22,0.15,0.5710,0.0187
9,DRS 10 - Piracicaba,2020-04-02,200,493679,384,0,384,0,119,142,0.15,0.4715,0.0408


In [168]:
if changeCSV and paramOpt==2:
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    dfparam['s0'] = dfparam['s0'].astype(float)
    dfparam['i0'] = dfparam['i0'].astype(float)
    dfparam['d0'] = dfparam['d0'].astype(float)
    dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','s0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','s0'].values[0]*1.1
    dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','i0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','i0']*.1
    dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','d0'] = dfparam.loc[dfparam.DRS=='DRS 14 - São João da Boa Vista','d0']*.5
    dfparam.loc[dfparam.DRS=='DRS 02 - Araçatuba','s0'] = dfparam.loc[dfparam.DRS=='DRS 02 - Araçatuba','s0']*3*2
    dfparam.loc[dfparam.DRS=='DRS 02 - Araçatuba','i0'] = dfparam.loc[dfparam.DRS=='DRS 02 - Araçatuba','i0']*.2*0.3
    dfparam.loc[dfparam.DRS=='DRS 02 - Araçatuba','d0'] = dfparam.loc[dfparam.DRS=='DRS 02 - Araçatuba','d0']*0.9
    dfparam.loc[dfparam.DRS=='DRS 03 - Araraquara','i0'] = dfparam.loc[dfparam.DRS=='DRS 03 - Araraquara','i0']*.2
    dfparam.loc[dfparam.DRS=='DRS 03 - Araraquara','d0'] = dfparam.loc[dfparam.DRS=='DRS 03 - Araraquara','d0']*.1
    dfparam.loc[dfparam.DRS=='DRS 04 - Baixada Santista','s0'] = dfparam.loc[dfparam.DRS=='DRS 04 - Baixada Santista','s0']*2.5*1.5
    dfparam.loc[dfparam.DRS=='DRS 05 - Barretos','i0'] = dfparam.loc[dfparam.DRS=='DRS 05 - Barretos','i0']*.1*0.5
    dfparam.loc[dfparam.DRS=='DRS 11 - Presidente Prudente','i0'] = dfparam.loc[dfparam.DRS=='DRS 11 - Presidente Prudente','i0']*.3
    dfparam.loc[dfparam.DRS=='DRS 10 - Piracicaba','i0'] = dfparam.loc[dfparam.DRS=='DRS 10 - Piracicaba','i0']*.3
    dfparam.loc[dfparam.DRS=='DRS 12 - Registro','d0'] = dfparam.loc[dfparam.DRS=='DRS 12 - Registro','d0']*.1
    dfparam.loc[dfparam.DRS=='DRS 12 - Registro','i0'] = dfparam.loc[dfparam.DRS=='DRS 12 - Registro','i0']*.1
    dfparam.loc[dfparam.DRS=='DRS 15 - São José do Rio Preto','i0'] = dfparam.loc[dfparam.DRS=='DRS 15 - São José do Rio Preto','i0']*.3
    dfparam.loc[dfparam.DRS=='DRS 16 - Sorocaba','i0'] = dfparam.loc[dfparam.DRS=='DRS 16 - Sorocaba','i0']*.3
    dfparam.loc[dfparam.DRS=='DRS 17 - Taubaté','i0'] = dfparam.loc[dfparam.DRS=='DRS 17 - Taubaté','i0']*.3        
    
if selectDRS and paramOpt==2:
    DRS=['DRS 14 - São João da Boa Vista','DRS 02 - Araçatuba','DRS 03 - Araraquara',
        'DRS 04 - Baixada Santista','DRS 05 - Barretos','DRS 11 - Presidente Prudente',
        'DRS 10 - Piracicaba','DRS 12 - Registro','DRS 16 - Sorocaba','DRS 17 - Taubaté']

if not selectDRS:
    DRS=dfparam.DRS    
    
if adjustParam:
    sCorrect=[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
    # sCorrect=[0.85,1.25,1.0,1.25,1.5,0.95,1.25,1.15,1.11,0.85,0.9]
    #103 [1.25,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.0,1.0,1.1]
    dfparam.s0=dfparam.s0.multiply(sCorrect[:len(states)], axis=0)

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)

if paramSave:
    dfparam.to_csv(paramFile)    

display(Markdown("## Modified Initial Parameters"))
display(dfparam)

## Modified Initial Parameters

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-18,200,7840120,136,0,136,300,196,117,0.15,0.4950,0.0081
1,DRS 02 - Araçatuba,2020-03-29,200,282699,182,0,182,0,51,5,0.15,0.6257,0.0362
2,DRS 03 - Araraquara,2020-03-30,200,377711,534,0,534,0,102,93,0.15,0.5056,0.0034
3,DRS 04 - Baixada Santista,2020-04-01,200,718013,270,0,270,0,281,155,0.15,0.4062,0.0124
4,DRS 05 - Barretos,2020-04-03,200,147064,883,0,883,0,14,105,0.15,0.3535,0.0211
5,DRS 06 - Bauru,2020-04-01,200,717995,263,0,263,0,201,59,0.15,0.5443,0.0304
6,DRS 07 - Campinas,2020-03-29,200,1865626,602,0,602,0,9,20,0.15,0.5461,0.0282
7,DRS 08 - Franca,2020-04-01,200,237187,37,0,37,0,26,3,0.15,0.4088,0.0257
8,DRS 09 - Marília,2020-03-30,200,456106,292,0,292,0,241,22,0.15,0.5710,0.0187
9,DRS 10 - Piracicaba,2020-04-02,200,493679,384,0,384,0,119,142,0.15,0.4715,0.0408


# Initial parameter optimization

In [169]:
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v2 as Learner #Yabox

In [170]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [171]:
allDistricts=True
cleanRecovered=False
version="8"

display(DRS)

['DRS 14 - São João da Boa Vista',
 'DRS 02 - Araçatuba',
 'DRS 03 - Araraquara',
 'DRS 04 - Baixada Santista',
 'DRS 05 - Barretos',
 'DRS 11 - Presidente Prudente',
 'DRS 10 - Piracicaba',
 'DRS 12 - Registro',
 'DRS 16 - Sorocaba',
 'DRS 17 - Taubaté']

In [ ]:
results=[]
if allDistricts:
    for districtRegion in DRS:
        query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        f=Learner.Learner.remote(districtRegion, *parameters, cleanRecovered, version)
        result = f.train.remote() 
        results.append(result)
else:
    districtRegion= "DRS 01 - Grande São Paulo" #'DRS 08 - Franca' \
    #'DRS 14 - São João da Boa Vista' #'DRS 04 - Baixada Santista' \
    #'DRS 11 - Presidente Prudente' #'DRS 13 - Ribeirão Preto' \
    #'DRS 05 - Barretos' #'DRS 12 - Registro' #'DRS 15 - São José do Rio Preto' \
    #'DRS 10 - Piracicaba'#'DRS 17 - Taubaté'#'DRS 02 - Araçatuba'# \
    #'DRS 03 - Araraquara' #DRS 07 - Campinas'#'DRS 16 - Sorocaba'#'DRS 06 - Bauru' \
    #'DRS 09 - Marília' #"DRS 01 - Grande São Paulo"
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    f=Learner.Learner.remote(districtRegion, *parameters, cleanRecovered, version)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

  0%|          | 91/6125000 [00:16<205:16:00,  8.29it/s]


  0%|          | 300/6125000 [00:30<92:56:53, 18.30it/s]


  0%|          | 703/6125000 [00:45<54:34:55, 31.17it/s]


  0%|          | 1830/6125000 [00:58<27:11:03, 62.57it/s]


  0%|          | 2775/6125000 [01:10<20:14:58, 83.98it/s]


...

  2%|▏         | 110215/6125000 [05:58<2:29:30, 670.51it/s]


  2%|▏         | 110215/6125000 [06:08<2:33:06, 654.73it/s]


  2%|▏         | 129286/6125000 [06:18<2:12:26, 754.48it/s]


  2%|▏         | 148785/6125000 [06:27<2:00:03, 829.59it/s]


  2%|▏         | 139128/6125000 [06:35<2:15:40, 735.30it/s]


In [ ]:
ray.shutdown()

In [ ]:
# Save data as csv
import glob
import os

path = './results/data'
files = glob.glob(os.path.join(path, "*.csv"))

df = (pd.read_csv(f).assign(DRS = f.split(" - ")[-1].split(".")[0]) for f in files)
df_all_drs = pd.concat(df, ignore_index=True)
df_all_drs.index.name = 'index'
df_all_drs.to_csv('./data/SEAIRD_sigmaOpt_AllDRS'+'.csv', sep=",")

# Plots

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots_v2 as covid_plots
import warnings
warnings.filterwarnings("ignore", message="Glyph 227 missing from current font.")
warnings.filterwarnings("ignore", message="Glyph 231 missing from current font.")

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

In [ ]:
#DRS 01 - Grande São Paulo
#DRS 02 - Araçatuba
#DRS 03 - Araraquara
#DRS 04 - Baixada Santista
#DRS 05 - Barretos
#DRS 06 - Bauru
#DRS 07 - Campinas
#DRS 08 - Franca
#DRS 09 - Marília
#DRS 10 - Piracicaba
#DRS 11 - Presidente Prudente
#DRS 12 - Registro
#DRS 13 - Ribeirão Preto
#DRS 14 - São João da Boa Vista
#DRS 15 - São José do Rio Preto
#DRS 16 - Sorocaba
#DRS 17 - Taubaté

#select districts for plotting
districts4Plot=['DRS 01 - Grande São Paulo',
               'DRS 04 - Baixada Santista',
               'DRS 07 - Campinas',
               'DRS 02 - Araçatuba',
               districtRegion]

#main district region for analysis
#districtRegion = "DRS 01 - Grande São Paulo"

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#number of cases to start plotting model in log graph - real data = 100
startCase=100

#all DRS for plotting
DRS=dfparam.DRS

In [ ]:
#do not allow the scrolling of the plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#plots one district or all districts
if opt==5:
    allDistricts=True
else:
    allDistricts=False
    districtRegion="DRS 01 - Grande São Paulo"
    
if allDistricts:
    for districtRegion in DRS:
        query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
        ratio = query['RATIO'][0]
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        
        #calcula data máxima dos gráficos
        #100 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate=dfSP.date.max()
        maxDate= datetime.strptime(lastDate, "%Y-%m-%d") + timedelta(days = 100) #"2020-08-31"
        maxDateStr = maxDate.strftime("%Y-%m-%d")

        covid_plots.covid_plots(districtRegion, districts4Plot, startdate, predict_range, \
                        startCase, opt, version, show=True, ratio=ratio, maxDate=maxDateStr, model=model)
else: 
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    ratio = query['RATIO'][0]
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    
    #calcula data máxima dos gráficos
    #100 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate=dfSP.date.max()
    maxDate= datetime.strptime(lastDate, "%Y-%m-%d") + timedelta(days = 100) #"2020-08-31"
    maxDateStr = maxDate.strftime("%Y-%m-%d")    
    
    covid_plots.covid_plots(districtRegion, districts4Plot, startdate, predict_range, \
                       startCase, opt, version, show=True, ratio=ratio, maxDate=maxDateStr, model=model)